# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [121]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [122]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


619

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

> **HINT:** The OpenWeatherMap API only allows 60 calls per minute under their free plan. Try using `time.sleep(60)` after each set of 50 cities to avoid API errors due to large calls.

In [123]:
# Setup for api calls
url = 'http://api.openweathermap.org/data/2.5/weather?'
#query_url = url + "appid=" + weather_api_key + "&q=" + city + '&units=imperial'

In [124]:
# for city in cities:
# c_data = requests.get(query_url).json()
# pprint(c_data)
city_data = []
counter = 0
for city in cities:
    counter = counter + 1
    if counter % 60 == 0:
        time.sleep(60)
        print('Taking a power nap')

    query_url = url + "appid=" + weather_api_key + "&q=" + city + '&units=imperial'
    c_data = requests.get(query_url).json()
    try:
        print(f"City {c_data['name']}  Temperture:  {c_data['main']['temp']}F Humidity: {c_data['main']['humidity']}%" +
        f" Clouds: {c_data['clouds']['all']}% Wind {c_data['wind']['speed']}mph")
        city_data.append({'name': c_data['name'],
                        'temp': c_data['main']['temp'],
                        'hum': c_data['main']['humidity'],
                        'clouds': c_data['clouds']['all'],
                        'wind': c_data['wind']['speed'],
                        'lat': c_data['coord']['lat']})
    except:
        print(f"Something croke {city}")

City Bluff  Temperture:  66.79F Humidity: 83% Clouds: 25% Wind 4.27mph
City Clyde River  Temperture:  -30.77F Humidity: 68% Clouds: 0% Wind 3.44mph
City Mabaruma  Temperture:  79.36F Humidity: 75% Clouds: 99% Wind 6.82mph
City Geraldton  Temperture:  68.41F Humidity: 64% Clouds: 13% Wind 12.66mph
City Nanortalik  Temperture:  26.71F Humidity: 88% Clouds: 100% Wind 15.66mph
City Rikitea  Temperture:  77.49F Humidity: 65% Clouds: 99% Wind 4.54mph
City Avarua  Temperture:  82.45F Humidity: 78% Clouds: 75% Wind 11.5mph
City George Town  Temperture:  78.57F Humidity: 84% Clouds: 20% Wind 1.99mph
City Kapaa  Temperture:  76.8F Humidity: 70% Clouds: 0% Wind 1.01mph
City Nikolskoye  Temperture:  24.15F Humidity: 97% Clouds: 100% Wind 5.06mph
Something croke marcona
City Iqaluit  Temperture:  -29.31F Humidity: 66% Clouds: 20% Wind 2.3mph
City Mataura  Temperture:  66.47F Humidity: 67% Clouds: 38% Wind 1.81mph
City Troitskaya  Temperture:  31.93F Humidity: 85% Clouds: 100% Wind 6.96mph
City Ushu

In [125]:
#pprint(city_data)
city_df = pd.DataFrame(city_data)
#pd.set_option('display.max_rows', 150)
city_df.to_csv(index=False)
city_df

,name,temp,hum,clouds,wind,lat
0,Bluff,66.79,83,25,4.27,-46.6000
1,Clyde River,-30.77,68,0,3.44,70.4692
2,Mabaruma,79.36,75,99,6.82,8.2000
3,Geraldton,68.41,64,13,12.66,-28.7667
4,Nanortalik,26.71,88,100,15.66,60.1432
...,...,...,...,...,...,...
561,Den Helder,40.91,76,40,17.00,52.9599
562,Salacgrīva,24.91,88,100,12.10,57.7536
563,Mayor Pablo Lagerenza,89.35,42,93,6.85,-19.9309
564,Vyazemskiy,-9.87,98,99,3.53,47.5378


In [126]:
# print(f"City {c_data['name']}  Temperture:  {c_data['main']['temp']}F Humidity: {c_data['main']['humidity']}%" +
#         f" Clouds: {c_data['clouds']['all']}% Wind {c_data['wind']['speed']}mph")

City Demyansk  Temperture:  28.85F Humidity: 98% Clouds: 100% Wind 3.06mph


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [127]:
city_df.to_csv("Output/city_data.csv", index=False, header=True)
city_df.head()

,name,temp,hum,clouds,wind,lat
0,Bluff,66.79,83,25,4.27,-46.6000
1,Clyde River,-30.77,68,0,3.44,70.4692
2,Mabaruma,79.36,75,99,6.82,8.2000
3,Geraldton,68.41,64,13,12.66,-28.7667
4,Nanortalik,26.71,88,100,15.66,60.1432


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [128]:
#  Get the indices of cities that have humidity over 100%.


In [129]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression